In [3]:
import os
import pandas as pd
import scipy.io as sio
import cv2
import numpy as np
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import confusion_matrix



name_save_directory = "LV_Net_Dice_Loss_Inner_with_Papilary_V2"
results = "_Results"
parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
saveFolder = os.path.join(parent_directory,name_save_directory)
name_save_results_directory = name_save_directory+results
path_segment_inner = os.path.join(saveFolder,name_save_results_directory)

name_save_directory = "LV_Net_Dice_Loss_Outer_V2"
results = "_Results"
parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
saveFolder = os.path.join(parent_directory,name_save_directory)
name_save_results_directory = name_save_directory+results
path_segment_outer = os.path.join(saveFolder,name_save_results_directory)

name_save_directory = "LV_Net_Dice_Loss_with_papilary_V2"
results = "_Results"
parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
saveFolder = os.path.join(parent_directory,name_save_directory)
name_save_results_directory = name_save_directory+results
path_segment_normal = os.path.join(saveFolder,name_save_results_directory)

name_save_directory = "LV_Net_Dice_Loss_Outer_Inner_with_Papilary_Final_Post_Processing_V2"
saving_metrics = 'Metrics_LV_Net_Dice_Loss_Outer_Inner_with_Papilary_Final_Post_Processing_V2.csv'

results = "_Results"
images = "_Joint_Images"
parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
saveFolder = os.path.join(parent_directory,name_save_directory)
# os.mkdir(saveFolder)

name_save_results_directory = name_save_directory+results
path_results_save = os.path.join(saveFolder,name_save_results_directory)
# os.mkdir(path_results_save)

name_save_images_directory = name_save_directory+images
path_images_save = os.path.join(saveFolder,name_save_images_directory)
# os.mkdir(path_images_save)


cols = ['sensitivity','specificity','accuracy','dice_score','Jaccard']
df = pd.DataFrame(columns=cols)
ratio=['Ratio']
dfr=pd.DataFrame(columns=ratio)
path_testing_ground_truth ="/tank/data/navchetan/Lars_Annotated_Datasets/Testing_Output_With_Papillary"

N_TESTING_SAMPLES = 342
print('Loading Testing Data for computation of Jaccard Index')
sensitivity_t = 0
specificity_t = 0
accuracy_t = 0
dice_score_t = 0
Jaccard_t = 0
countoutin = 0
for i in range(N_TESTING_SAMPLES):
    if(i%10 == 0):
        print(i)
    os.chdir(path_segment_outer)    
    pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
    x = sio.loadmat(pathr)
    Segment_model = x['S']
    Segment_model1 = Segment_model*255;
    retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
    Segment_model_threshold_out = Segment_model_threshold/255.0
    
    os.chdir(path_segment_inner)    
    pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
    x = sio.loadmat(pathr)
    Segment_model = x['S']
    Segment_model1 = Segment_model*255;
    retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
    Segment_model_threshold_in = Segment_model_threshold/255.0
    
    Segment_model_threshold1=Segment_model_threshold_out-Segment_model_threshold_in
    
    for k in range(256):
        for l in range(256):
            if Segment_model_threshold1[k,l]<0:
                Segment_model_threshold1[k,l]=0
                
    
    os.chdir(path_segment_normal)    
    pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
    x = sio.loadmat(pathr)
    Segment_model = x['S']
    Segment_model1 = Segment_model*255;
    retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
    Segment_outin = Segment_model_threshold/255.0
    
#     kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)) 
#     Segment_outin= cv2.morphologyEx(Segment_outin, cv2.MORPH_CLOSE, kernel)
#     Segment_outin = cv2.morphologyEx(Segment_outin, cv2.MORPH_OPEN, kernel)
#     Segment_outin= cv2.morphologyEx(Segment_outin, cv2.MORPH_CLOSE, kernel)
#     Segment_outin = cv2.morphologyEx(Segment_outin, cv2.MORPH_OPEN, kernel)
    
    for m in range(256):
        for n in range(256):
            if Segment_model_threshold1[m,n]==1 and Segment_outin[m,n]==0:
                Segment_model_threshold1[m,n]=1
            elif Segment_model_threshold1[m,n]==0 and Segment_outin[m,n]==1:
                Segment_model_threshold1[m,n]=1
            elif Segment_model_threshold1[m,n]==0 and Segment_outin[m,n]==0:
                Segment_model_threshold1[m,n]=0
# chnaging form here
#     for m in range(256):
#         for n in range(256):
#             if Segment_model_threshold1[m,n]==1 and Segment_model_threshold_out[m,n]==0:
#                 Segment_model_threshold1[m,n]=0
#             elif Segment_model_threshold1[m,n]==0 and Segment_model_threshold_out[m,n]==1:
#                 Segment_model_threshold1[m,n]=0
#             elif Segment_model_threshold1[m,n]==0 and Segment_model_threshold_out[m,n]==0:
#                 Segment_model_threshold1[m,n]=0
#             elif Segment_model_threshold1[m,n]==1 and Segment_model_threshold_out[m,n]==1:
#                 Segment_model_threshold1[m,n]=1
# upto here
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))            
    Segment_model_threshold1= cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_CLOSE, kernel)
    Segment_model_threshold1 = cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_OPEN, kernel)
    Segment_model_threshold1= cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_CLOSE, kernel)
    Segment_model_threshold1 = cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_OPEN, kernel)    
    
    os.chdir(path_results_save)
    S = Segment_model_threshold1
    countoutin = countoutin + 1
    fsave = 'Segment_Output'+str(countoutin).zfill(5)+'.mat'
    sio.savemat(fsave, {'S':S})
            
    os.chdir(path_testing_ground_truth)
    path_t = 'Segment'+str(i+1).zfill(5)+'.mat'
    y = sio.loadmat(path_t)
    ground_truth_testing  = y['S']
    ground_truth1 = ground_truth_testing*255;
    retval2, ground_truth_testing_threshold = cv2.threshold(ground_truth1,0,255,cv2.THRESH_BINARY)
    ground_truth_testing_threshold1 = ground_truth_testing_threshold/255.0
    
    mcm = confusion_matrix(np.ndarray.flatten(Segment_model_threshold1),np.ndarray.flatten(ground_truth_testing_threshold1))
    tn = mcm[0, 0]
    fp = mcm[0, 1]
    fn = mcm[1, 0]
    tp = mcm[1, 1]
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fn + fp)
    dice_score = 2*tp / (2*tp + fp + fn)
    Jaccard = dice_score / (2-dice_score)
    
    sensitivity_t = sensitivity_t + sensitivity
    specificity_t = specificity_t + specificity
    accuracy_t = accuracy_t + accuracy
    dice_score_t = dice_score_t + dice_score
    Jaccard_t = Jaccard_t + Jaccard
    
    
    df = df.append(pd.Series([sensitivity,specificity,accuracy,dice_score,Jaccard],index=df.columns), ignore_index=True)

os.chdir(saveFolder)
df.to_csv(saving_metrics,index=False)
#dfr.to_csv('Ratio.csv')
print(sensitivity_t/N_TESTING_SAMPLES)
print(specificity_t/N_TESTING_SAMPLES)
print(dice_score_t/N_TESTING_SAMPLES)
print(accuracy_t/N_TESTING_SAMPLES)
print(Jaccard_t/N_TESTING_SAMPLES)

Loading Testing Data for computation of Jaccard Index
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
0.8683130749704316
0.9848000592301783
0.9024017795154323
0.9601278918528418
0.8233202517222761


In [2]:
print(np.std(df.sensitivity))
print(np.std(df.specificity))
print(np.std(df.dice_score))
print(np.std(df.accuracy))
print(np.std(df.Jaccard))

0.05215239732944125
0.010436276211473335
0.027999463533178958
0.013294337815073737
0.04549394700138018


In [3]:
O1 = np.zeros((N_TESTING_SAMPLES,256,256))
O2 = np.zeros((N_TESTING_SAMPLES,256,256))
Ground_Truth = np.zeros((N_TESTING_SAMPLES,256,256))
Segmentation_from_model_out = np.zeros((N_TESTING_SAMPLES,256,256))
Segmentation_from_model_in = np.zeros((N_TESTING_SAMPLES,256,256))
Segmentation_from_model = np.zeros((N_TESTING_SAMPLES,256,256))

path = "/tank/data/navchetan/Lars_Annotated_Datasets/Testing_Input"
os.chdir(path)
for i in range(N_TESTING_SAMPLES):
    if(i%10 == 0):
        print(i)
    pathr = 'Image'+str(i+1).zfill(5)+'.mat'
    x = sio.loadmat(pathr)
    O1[i,:,:] = x['U']/255
    O2[i,:,:] = x['U']/255
    
path = "/tank/data/navchetan/Lars_Annotated_Datasets/Testing_Output_With_Papillary"
os.chdir(path)
for i in range(N_TESTING_SAMPLES):
    if(i%10 == 0):
        print(i)
    pathr = 'Segment'+str(i+1).zfill(5)+'.mat'
    x = sio.loadmat(pathr)
    Ground_Truth[i,:,:] = x['S']
    retval2, ground_truth_testing_threshold = cv2.threshold(Ground_Truth[i,:,:],0,255,cv2.THRESH_BINARY)
    Ground_Truth[i,:,:] = ground_truth_testing_threshold/255.0
    

os.chdir(path_results_save)
for i in range(N_TESTING_SAMPLES):
    if(i%10 == 0):
        print(i)
    pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
    x = sio.loadmat(pathr)
    Segmentation_from_model[i,:,:] = x['S']
    Segment_model1 = Segmentation_from_model[i,:,:]*255;
    retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
    Segmentation_from_model[i,:,:] = Segment_model_threshold/255.0
    
def concat_images(imga, imgb):
    """
    Combines two color image ndarrays side-by-side.
    """
    [ha,wa] = np.shape(imga)
    [hb,wb] = np.shape(imgb)
    max_height = np.max([ha, hb])
    total_width = wa+wb
    new_img = np.zeros(shape=(max_height, total_width))
    new_img[:ha,:wa]=imga
    new_img[:hb,wa:wa+wb]=imgb
    return new_img


h = 256
w = 256
img_array = []    
for k in range(N_TESTING_SAMPLES):
    print(k)
    for i in range(h):
        for j in range (w):
            if Ground_Truth[k,i,j]>0:
                O1[k,i,j]=Ground_Truth[k,i,j]
            if Segmentation_from_model[k,i,j]>0:
                O2[k,i,j]=Segmentation_from_model[k,i,j]
    W1 = O1[k,:,:]
    W2 = O2[k,:,:]
    img = concat_images(W1,W2)
    img_array.append(img)
    
    os.chdir(path_images_save) 
    pathr = 'Joint_Image'+str(k+1).zfill(5)+'.png'
    cv2.imwrite(pathr, img*255) 

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180

In [4]:
# import os
# import pandas as pd
# import scipy.io as sio
# import cv2
# import numpy as np
# from sklearn.metrics import jaccard_similarity_score
# from sklearn.metrics import confusion_matrix



# name_save_directory = "LV_Net_Dice_Loss_Inner_with_Papilary_V2"
# results = "_Results"
# parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
# saveFolder = os.path.join(parent_directory,name_save_directory)
# name_save_results_directory = name_save_directory+results
# path_segment_inner = os.path.join(saveFolder,name_save_results_directory)

# name_save_directory = "LV_Net_Dice_Loss_Outer_V2"
# results = "_Results"
# parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
# saveFolder = os.path.join(parent_directory,name_save_directory)
# name_save_results_directory = name_save_directory+results
# path_segment_outer = os.path.join(saveFolder,name_save_results_directory)

# name_save_directory = "LV_Net_Dice_Loss_with_papilary_V2"
# results = "_Results"
# parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
# saveFolder = os.path.join(parent_directory,name_save_directory)
# name_save_results_directory = name_save_directory+results
# path_segment_normal = os.path.join(saveFolder,name_save_results_directory)

# name_save_directory = "LV_Net_Dice_Loss_Outer_Inner_with_Papilary_Final_Post_Processing_V2"
# saving_metrics = 'Metrics_LV_Net_Dice_Loss_Outer_Inner_with_Papilary_Final_Post_Processing_V2.csv'

# results = "_Results"
# images = "_Joint_Images"
# parent_directory = "/tank/data/navchetan/Lars_Annotated_Datasets/Results/"
# saveFolder = os.path.join(parent_directory,name_save_directory)
# # os.mkdir(saveFolder)

# name_save_results_directory = name_save_directory+results
# path_results_save = os.path.join(saveFolder,name_save_results_directory)
# # os.mkdir(path_results_save)

# name_save_images_directory = name_save_directory+images
# path_images_save = os.path.join(saveFolder,name_save_images_directory)
# # os.mkdir(path_images_save)


# cols = ['sensitivity','specificity','accuracy','dice_score','Jaccard']
# df = pd.DataFrame(columns=cols)
# ratio=['Ratio']
# dfr=pd.DataFrame(columns=ratio)
# path_testing_ground_truth ="/tank/data/navchetan/Lars_Annotated_Datasets/Testing_Output_With_Papillary"

# N_TESTING_SAMPLES = 342
# print('Loading Testing Data for computation of Jaccard Index')
# sensitivity_t = 0
# specificity_t = 0
# accuracy_t = 0
# dice_score_t = 0
# Jaccard_t = 0
# countoutin = 0

In [5]:
# for i in range(N_TESTING_SAMPLES):
#     if(i%10 == 0):
#         print(i)

In [6]:
# i=260

In [7]:
#     import matplotlib.pyplot as plt
# import os
# import matplotlib.pyplot as plt
#     os.chdir(path_segment_outer)    
#     pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
#     x = sio.loadmat(pathr)
#     Segment_model = x['S']
#     Segment_model1 = Segment_model*255;
#     retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
#     Segment_model_threshold_out = Segment_model_threshold/255.0
#     plt.imshow(Segment_model_threshold_out, cmap="gray")

In [8]:
#     os.chdir(path_segment_inner)    
#     pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
#     x = sio.loadmat(pathr)
#     Segment_model = x['S']
#     Segment_model1 = Segment_model*255;
#     retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
#     Segment_model_threshold_in = Segment_model_threshold/255.0
#     plt.imshow(Segment_model_threshold_in, cmap="gray")

In [9]:
#     Segment_model_threshold1=Segment_model_threshold_out-Segment_model_threshold_in
    
#     for i in range(256):
#         for j in range(256):
#             if Segment_model_threshold1[i,j]<0:
#                 Segment_model_threshold1[i,j]=0
                
#     plt.imshow(Segment_model_threshold1, cmap="gray")    

In [10]:
#     os.chdir(path_segment_normal)    
#     pathr = 'Segment_Output'+str(i+1).zfill(5)+'.mat'
#     x = sio.loadmat(pathr)
#     Segment_model = x['S']
#     Segment_model1 = Segment_model*255;
#     retval1, Segment_model_threshold = cv2.threshold(Segment_model1,100,255,cv2.THRESH_BINARY)
#     Segment_outin = Segment_model_threshold/255.0
#     plt.imshow(Segment_outin, cmap="gray")        

In [11]:
# i

In [12]:
# for i in range(256):
#         for j in range(256):
#             if Segment_model_threshold1[i,j]==1 and Segment_outin[i,j]==0:
#                 Segment_model_threshold1[i,j]=1
#             elif Segment_model_threshold1[i,j]==0 and Segment_outin[i,j]==1:
#                 Segment_model_threshold1[i,j]=1
#             elif Segment_model_threshold1[i,j]==0 and Segment_outin[i,j]==0:
#                 Segment_model_threshold1[i,j]=0
# kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))            
# Segment_model_threshold1= cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_CLOSE, kernel)
# Segment_model_threshold1 = cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_OPEN, kernel)
# Segment_model_threshold1= cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_CLOSE, kernel)
# Segment_model_threshold1 = cv2.morphologyEx(Segment_model_threshold1, cv2.MORPH_OPEN, kernel)    
    
# plt.imshow(Segment_model_threshold1, cmap="gray")     

In [13]:
#     os.chdir(path_results_save)
#     S = Segment_model_threshold1
#     countoutin = countoutin + 1
#     fsave = 'Segment_Output'+str(countoutin).zfill(5)+'.mat'
#     sio.savemat(fsave, {'S':S})
            
#     os.chdir(path_testing_ground_truth)
#     path_t = 'Segment'+str(i+1).zfill(5)+'.mat'
#     y = sio.loadmat(path_t)
#     ground_truth_testing  = y['S']
#     ground_truth1 = ground_truth_testing*255;
#     retval2, ground_truth_testing_threshold = cv2.threshold(ground_truth1,0,255,cv2.THRESH_BINARY)
#     ground_truth_testing_threshold1 = ground_truth_testing_threshold/255.0
    
#     mcm = confusion_matrix(np.ndarray.flatten(Segment_model_threshold1),np.ndarray.flatten(ground_truth_testing_threshold1))
#     tn = mcm[0, 0]
#     fp = mcm[0, 1]
#     fn = mcm[1, 0]
#     tp = mcm[1, 1]
    
#     sensitivity = tp / (tp + fn)
#     specificity = tn / (tn + fp)
#     accuracy = (tp + tn) / (tp + tn + fn + fp)
#     dice_score = 2*tp / (2*tp + fp + fn)
#     Jaccard = dice_score / (2-dice_score)
    
#     sensitivity_t = sensitivity_t + sensitivity
#     specificity_t = specificity_t + specificity
#     accuracy_t = accuracy_t + accuracy
#     dice_score_t = dice_score_t + dice_score
#     Jaccard_t = Jaccard_t + Jaccard
    
    
#     df = df.append(pd.Series([sensitivity,specificity,accuracy,dice_score,Jaccard],index=df.columns), ignore_index=True)

# os.chdir(saveFolder)
# df.to_csv(saving_metrics,index=False)
# #dfr.to_csv('Ratio.csv')
# print(sensitivity_t/N_TESTING_SAMPLES)
# print(specificity_t/N_TESTING_SAMPLES)
# print(dice_score_t/N_TESTING_SAMPLES)
# print(accuracy_t/N_TESTING_SAMPLES)
# print(Jaccard_t/N_TESTING_SAMPLES)